In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Tạo dữ liệu điểm số (Score)
data = [7, 8, 8, 7.5, 9, 10, 6, 7, 7, 8, 6, 5, 20]  # 20 là ngoại lệ

df = pd.DataFrame(data, columns=["Score"])

# Tạo kết nối SQLite và lưu vào bảng
conn = sqlite3.connect(":memory:")
df.to_sql("scores", conn, index=False, if_exists="replace")


13

In [2]:
median = df["Score"].median()
df["abs_dev"] = abs(df["Score"] - median)
mad = df["abs_dev"].median()
threshold = 1.5 * mad


In [4]:
# Lưu vào bảng SQLite để có thể dùng SQL lọc
df.to_sql("scores_with_dev", conn, index=False, if_exists="replace")

# Viết truy vấn SQL để lọc các điểm ngoại lệ
query = f"""
SELECT *
FROM scores_with_dev
WHERE abs(Score - {median}) > {threshold}
"""

outliers = pd.read_sql_query(query, conn)
print(" Các giá trị ngoại lệ theo MAD:")
print(outliers)


 Các giá trị ngoại lệ theo MAD:
   Score  abs_dev
0    9.0      1.5
1   10.0      2.5
2    6.0      1.5
3    6.0      1.5
4    5.0      2.5
5   20.0     12.5


MAD = Median của |x - median(X)|

Tính median(Score):
Sau khi sắp xếp: [5.0, 6.0, 6.0, 9.0, 10.0, 20.0]

Trung vị = (6.0 + 9.0) / 2 = 7.5

Tính abs_dev = |Score - median|

Tính MAD = median(abs_dev):
abs_dev = [1.5, 2.5, 1.5, 1.5, 2.5, 12.5]

Sắp xếp: [1.5, 1.5, 1.5, 2.5, 2.5, 12.5]

Median = (1.5 + 2.5)/2 = 2.0

Áp dụng ngưỡng ngoại lệ:
Ngưỡng = 1.5 * MAD = 1.5 * 2.0 = 3.0

Kiểm tra:
Chỉ có duy nhất:

|20 - 7.5| = 12.5 > 3.0
→ Score = 20.0 bị đánh dấu là ngoại lệ

Chỉ có Score = 20.0 là ngoại lệ theo phương pháp MAD với hệ số 1.5.